# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).
You need an API key - which you can get for free - to use this notebook!

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [1]:
from datetime import datetime
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
LIMIT = 1000  # only load the top 1000 currencies, if you have invested in smaller ones increase this limit
API_KEY = ""  # add your API key here

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':LIMIT,
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_KEY,
}

session = Session()
session.headers.update(headers)

try:
    # load the json data from the API
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
except Exception as e:
    print(e)
if data["status"]["error_message"]:
    print(data["status"]["error_message"])

In [3]:
# transform data using pandas and keep track of the time
now = datetime.now()
market_data = pd.DataFrame(data['data'])
market_data.head(10)
market_data["price_usd"] = market_data.quote.apply(lambda x: x['USD']['price'])
market_data.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,last_updated,quote,price_usd
0,1,Bitcoin,BTC,bitcoin,9893,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.865906e+07,1.865906e+07,None,1,2021-03-20T11:13:02.000Z,"{'USD': {'price': 58976.32978384015, 'volume_2...",58976.329784
1,1027,Ethereum,ETH,ethereum,6247,2015-08-07T00:00:00.000Z,"[mineable, pow, smart-contracts, ethereum, coi...",NaN,1.151261e+08,1.151261e+08,"{'id': 2502, 'name': 'Heco', 'symbol': 'HT', '...",2,2021-03-20T11:13:02.000Z,"{'USD': {'price': 1842.005493712207, 'volume_2...",1842.005494
2,1839,Binance Coin,BNB,binance-coin,604,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",1.705328e+08,1.545328e+08,1.705328e+08,None,3,2021-03-20T11:12:08.000Z,"{'USD': {'price': 270.2093550845516, 'volume_2...",270.209355
3,2010,Cardano,ADA,cardano,260,2017-10-01T00:00:00.000Z,"[mineable, dpos, pos, platform, research, smar...",4.500000e+10,3.194831e+10,4.500000e+10,None,4,2021-03-20T11:12:10.000Z,"{'USD': {'price': 1.27283977624264, 'volume_24...",1.272840
4,825,Tether,USDT,tether,11564,2015-02-25T00:00:00.000Z,"[store-of-value, payments, stablecoin, stablec...",NaN,3.943707e+10,4.063629e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",5,2021-03-20T11:12:09.000Z,"{'USD': {'price': 1.0032891986942, 'volume_24h...",1.003289


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [4]:
# enter your coins below:
block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'NANO': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,BTC,0.31337
1,BCH,0.01337
2,ETH,3.00000
3,LTC,1.50000
4,MIOTA,777.00000
5,XRP,13.00000
6,DASH,0.10000
7,XMR,1.00000
8,LSK,12.00000
9,OMG,1.10000


### Combining the Blockfolio and the API Data

In [5]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.total_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data

,symbol,amount,id,name,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,last_updated,quote,price_usd,value_usd,coinshare
0,BTC,0.31337,1,Bitcoin,bitcoin,9893,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.865906e+07,1.865906e+07,None,1,2021-03-20T11:13:02.000Z,"{'USD': {'price': 58976.32978384015, 'volume_2...",58976.329784,18481.412464,1.679452e-08
2,ETH,3.00000,1027,Ethereum,ethereum,6247,2015-08-07T00:00:00.000Z,"[mineable, pow, smart-contracts, ethereum, coi...",NaN,1.151261e+08,1.151261e+08,"{'id': 2502, 'name': 'Heco', 'symbol': 'HT', '...",2,2021-03-20T11:13:02.000Z,"{'USD': {'price': 1842.005493712207, 'volume_2...",1842.005494,5526.016481,2.605839e-08
4,MIOTA,777.00000,1720,IOTA,iota,71,2017-06-13T00:00:00.000Z,"[dag, medium-of-exchange, iot, sharing-economy]",2.779530e+09,2.779530e+09,2.779530e+09,None,28,2021-03-20T11:12:07.000Z,"{'USD': {'price': 1.45300671164127, 'volume_24...",1.453007,1128.986215,2.795436e-07
13,NEO,7.00000,1376,Neo,neo,276,2016-09-08T00:00:00.000Z,"[platform, enterprise-solutions, smart-contracts]",1.000000e+08,7.053883e+07,1.000000e+08,None,38,2021-03-20T11:13:04.000Z,"{'USD': {'price': 45.24686276665593, 'volume_2...",45.246863,316.728039,7.000000e-08
15,STEEM,499.00000,1230,Steem,steem,36,2016-04-18T00:00:00.000Z,"[media, content-creation]",NaN,3.762131e+08,3.931872e+08,None,171,2021-03-20T11:13:04.000Z,"{'USD': {'price': 0.63170196647362, 'volume_24...",0.631702,315.219281,1.269116e-06
3,LTC,1.50000,2,Litecoin,litecoin,769,2013-04-28T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, bi...",8.400000e+07,6.669300e+07,6.669300e+07,"{'id': 2502, 'name': 'Heco', 'symbol': 'HT', '...",9,2021-03-20T11:13:02.000Z,"{'USD': {'price': 204.43432842063612, 'volume_...",204.434328,306.651493,2.249112e-08
7,XMR,1.00000,328,Monero,monero,184,2014-05-21T00:00:00.000Z,"[mineable, pow, medium-of-exchange, ringct]",NaN,1.786697e+07,1.786697e+07,None,25,2021-03-20T11:13:05.000Z,"{'USD': {'price': 236.49034204395596, 'volume_...",236.490342,236.490342,5.596919e-08
10,NANO,42.00000,1567,Nano,nano,56,2017-03-06T00:00:00.000Z,"[pow, blake2b, medium-of-exchange, payments]",1.332483e+08,1.332483e+08,1.332483e+08,None,98,2021-03-20T11:13:06.000Z,"{'USD': {'price': 5.2707062945197, 'volume_24h...",5.270706,221.369664,3.152010e-07
11,PIVX,123.00000,1169,PIVX,pivx,24,2016-02-13T00:00:00.000Z,"[pos, medium-of-exchange, store-of-value, priv...",NaN,6.518059e+07,6.518059e+07,None,373,2021-03-20T11:13:03.000Z,"{'USD': {'price': 1.47495017551526, 'volume_24...",1.474950,181.418872,1.887065e-06
8,LSK,12.00000,1214,Lisk,lisk,60,2016-04-06T00:00:00.000Z,"[dpos, platform, enterprise-solutions, smart-c...",NaN,1.275609e+08,1.434961e+08,None,116,2021-03-20T11:13:04.000Z,"{'USD': {'price': 4.12189426343539, 'volume_24...",4.121894,49.462731,8.362597e-08


### Your Blockfolio in numbers and graphs!


In [6]:
networth = 'Your blockfolio is currently (i.e {}) worth {:.2f} USD!'.format(now.strftime('%Y-%m-%d %I:%M %p'), 
                                                                        merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently (i.e 2021-03-20 12:13 PM) worth 26844.18 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [7]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [8]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)